<a href="https://colab.research.google.com/github/cagBRT/PointCloud/blob/main/Depth_Maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/cagBRT/PointCloud.git cloned-repo
%cd cloned-repo

In [ ]:
from IPython.display import Image

Albumentations is a Python library for performing data augmentation for computer vision. It supports various computer vision tasks such as image classification, object detection, segmentation, and keypoint estimation.

In [ ]:
!pip install -U albumentations

In [ ]:
!pip install datasets

In [ ]:
Image("distanceImage.png", width=400)

The dataset has two fields:

image: a PIL PNG image object with uint8 data type.
depth_map: a PIL Tiff image object with float32 data type which is the depth map of the image.

In [ ]:
train_dataset2 = load_dataset("sayakpaul/nyu_depth_v2", split="validation")

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset("sayakpaul/nyu_depth_v2", split="val")


In [ ]:
example["image"]

Before we look at the depth map, we need to first convert its data type to uint8 using .convert('RGB') as PIL can’t display float32 images. Now take a look at its corresponding depth map:

In [ ]:
example["depth_map"].convert("RGB")


It’s all black! You’ll need to add some color to the depth map to visualize it properly. To do that, either we can apply color automatically during display using plt.imshow() or create a colored depth map using plt.cm and then display it. In this example, we have used the latter one, as we can save/write the colored depth map later. (the utility below is taken from the FastDepth repository).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

cmap = plt.cm.viridis

def colored_depthmap(depth, d_min=None, d_max=None):
    if d_min is None:
        d_min = np.min(depth)
    if d_max is None:
        d_max = np.max(depth)
    depth_relative = (depth - d_min) / (d_max - d_min)
    return 255 * cmap(depth_relative)[:,:,:3]

def show_depthmap(depth_map):
   if not isinstance(depth_map, np.ndarray):
       depth_map = np.array(depth_map)
   if depth_map.ndim == 3:
       depth_map = depth_map.squeeze()

   d_min = np.min(depth_map)
   d_max = np.max(depth_map)
   depth_map = colored_depthmap(depth_map, d_min, d_max)

   plt.imshow(depth_map.astype("uint8"))
   plt.axis("off")
   plt.colorbar()
   plt.show()

In [ ]:
show_depthmap(example["depth_map"])

In [ ]:
example2 = train_dataset[88]
example2["image"]

In [ ]:
show_depthmap(example2["depth_map"])

You can also visualize several different images and their corresponding depth maps.

In [ ]:
def merge_into_row(input_image, depth_target):
    if not isinstance(input_image, np.ndarray):
        input_image = np.array(input_image)
    d_min = np.min(depth_target)
    d_max = np.max(depth_target)
    depth_target_col = colored_depthmap(depth_target, d_min, d_max)
    img_merge = np.hstack([input_image, depth_target_col])
    return img_merge

random_indices = np.random.choice(len(train_dataset), 9).tolist()
plt.figure(figsize=(15, 6))
for i, idx in enumerate(random_indices):
    example = train_dataset[idx]
    ax = plt.subplot(3, 3, i + 1)
    image_viz = merge_into_row(
        example["image"], example["depth_map"]
    )
    plt.imshow(image_viz.astype("uint8"))
    plt.axis("off")